Inference\
https://wikidocs.net/115055

https://huggingface.co/docs/transformers/index

http://kkma.snu.ac.kr/documents/index.jsp?doc=postag


# Author: Yoonhyuck WOO / JBNU_Industrial Information system Engineering
# Date; 2. 10. 2022 - 2. 22. 2022
# Title: Korean_NER
# Professor: Seung-Hoon Na

In [103]:
import os
import pandas as pd
import json
import transformers
from transformers import BertTokenizer
# from tqdm.auto import tqdm

In [2]:
PATH_dir = 'C:\\Users\\LG\\Desktop\\github\\JBNU-2022-SPRING\\English world class tagging & Korean_Named Entity Recognition\\Ko_En_NER_POStag_data\Ko_NER_POS'
file_name_ko_train = 'train.txt'
file_name_ko_test = 'test.txt'
file_name_ko_dev = 'dev.txt'

In [140]:
file_name = file_name_ko_dev
PATH_data = os.path.join(PATH_dir, file_name)

# change file extention from '.txt' to '.json'
prepro_file_name = 'prepro_' + file_name.rsplit('.')[0] + '.json'
PATH_preprodata = os.path.join(PATH_dir, prepro_file_name)

In [141]:
print(PATH_data)

C:\Users\LG\Desktop\github\JBNU-2022-SPRING\English world class tagging & Korean_Named Entity Recognition\Ko_En_NER_POStag_data\Ko_NER_POS\dev.txt


In [142]:
print(file_name.rsplit('.')[1])

txt


In [143]:
print(prepro_file_name)

prepro_dev.json


In [144]:
print(PATH_preprodata)

C:\Users\LG\Desktop\github\JBNU-2022-SPRING\English world class tagging & Korean_Named Entity Recognition\Ko_En_NER_POStag_data\Ko_NER_POS\prepro_dev.json


In [145]:
with open(PATH_data, 'r', encoding='UTF8') as f:
    rawdata = f.readlines()

In [146]:
df0 = pd.DataFrame(rawdata)
df0

,0
0,선방/NNG\tNNG\tB\tO\n
1,한/XSV~ETM\tXSV~ETM\tI\tO\n
2,정성룡/NNP\tNNP\tB\tB-PS\n
3,이/JKS\tJKS\tI\tO\n
4,볼/NNG\tNNG\tB\tO\n
...,...
6940,문화/NNG\tNNG\tB\tO\n
6941,체육/NNG\tNNG\tI\tO\n
6942,센터/NNG\tNNG\tI\tO\n
6943,)/SS\tSS\tI\tO\n


In [147]:
df = pd.DataFrame(df0[0].str.split('/').tolist(),columns=['1-1','1-2'])
df

,1-1,1-2
0,선방,NNG\tNNG\tB\tO\n
1,한,XSV~ETM\tXSV~ETM\tI\tO\n
2,정성룡,NNP\tNNP\tB\tB-PS\n
3,이,JKS\tJKS\tI\tO\n
4,볼,NNG\tNNG\tB\tO\n
...,...,...
6940,문화,NNG\tNNG\tB\tO\n
6941,체육,NNG\tNNG\tI\tO\n
6942,센터,NNG\tNNG\tI\tO\n
6943,),SS\tSS\tI\tO\n


In [148]:
df2 = pd.DataFrame(df0[0].str.split('\t').tolist(),columns=['1','2','3','4'])
df2

,1,2,3,4
0,선방/NNG,NNG,B,O\n
1,한/XSV~ETM,XSV~ETM,I,O\n
2,정성룡/NNP,NNP,B,B-PS\n
3,이/JKS,JKS,I,O\n
4,볼/NNG,NNG,B,O\n
...,...,...,...,...
6940,문화/NNG,NNG,B,O\n
6941,체육/NNG,NNG,I,O\n
6942,센터/NNG,NNG,I,O\n
6943,)/SS,SS,I,O\n


In [149]:
df7 = pd.concat([df['1-1'],df2['4'].str.strip('\n')],axis = 1)
df7 = pd.DataFrame(df7)
df7 = df7.rename(columns={'1-1':'Entity','4':'tag'})
df7

,Entity,tag
0,선방,O
1,한,O
2,정성룡,B-PS
3,이,O
4,볼,O
...,...,...
6940,문화,O
6941,체육,O
6942,센터,O
6943,),O


In [150]:
print(len(df7))

6945


# Preprocessing 1

In [189]:
lst=[]
lst_set=[]
lst_ent=[]
lst_tag=[]
for i in range(6945):
    if df7['tag'][i] != None:
        lst_ent.append(df7['Entity'][i])
        lst_tag.append(df7['tag'][i])
    else:
        lst_set.append(lst_ent)
        lst_set.append(lst_tag)
        lst.append(lst_set)
        lst_ent = []
        lst_tag = []
        lst_set = [] 

In [192]:
print(lst[0][0])
print(lst[0][1])

['선방', '한', '정성룡', '이', '볼', '을', '걷', '어', '내', '고', '있', '다', '.']
['O', 'O', 'B-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [202]:
print(len(df7['Entity'][0]))
print(df7['Entity'][0])
morph_to_tokens = tokenizer.tokenize(df7['Entity'][0])
print(morph_to_tokens)
print(len(morph_to_tokens))

2
선방
['선', '##방']
2


# Preprocessing 2

만약, BERT의 단어 집합에 em, ##bed, ##ding, #s라는 서브 워드들이 존재한다면, embeddings는 em, ##bed, ##ding, #s로 분리됩니다. 여기서 ##은 이 서브워드들은 단어의 중간부터 등장하는 서브워드라는 것을 알려주기 위해 단어 집합 생성 시 표시해둔 기호입니다. 이런 표시가 있어야만 em, ##bed, ##ding, #s를 다시 손쉽게 embeddings로 복원할 수 있을 것

In [153]:
PRETAINED_MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(PRETAINED_MODEL_NAME)

In [194]:
lst=[]
lst_set = []
lst_ent = []
lst_tag = []
for i in range(6945):
    if df7['tag'][i] != None:
        lst_ent.append(df7['Entity'][i])
        lst_tag.append(df7['tag'][i])

    else:
        lst_set.append(lst_ent)
        result = ' '.join(str(s) for s in lst_ent)
        morph_to_tokens = tokenizer.tokenize(result)
        lst_set.append(morph_to_tokens)
        lst_set.append(lst_tag)
        lst.append(lst_set)
        result = 0
        lst_ent = []
        lst_tok = []
        lst_tag = []
        lst_set = [] 

In [195]:
print('entity', len(lst[0][0]))
print(lst[0][0])

print('token', len(lst[0][1]))
print(lst[0][1])

print('entity_tag', len(lst[0][2]))
print(lst[0][2])

# print('tok_tag', len(lst[3][]))
# print(lst[3][])

entity 13
['선방', '한', '정성룡', '이', '볼', '을', '걷', '어', '내', '고', '있', '다', '.']
token 16
['선', '##방', '한', '정', '##성', '##룡', '이', '볼', '을', '걷', '어', '내', '고', '있', '다', '.']
entity_tag 13
['O', 'O', 'B-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [170]:
print(len(lst[3][0][2]))
print(lst[3][0])


1
29 분 마이클 캐릭 의 침투 패스 를 오른쪽 측면 에서 이어받 은 박지성 은 절묘 한 개인기 로 미들즈브러 수비수 를 제친 뒤 골문 으로 강하 고 정확 한 패스 를 넣 었 고 루니 가 이 를 오른발 슛 으로 연결 , 승부 를 원점 으로 돌렸 다 .


# Tokenizer tagging

In [200]:
lst=[]
lst_set = []
lst_ent = []
lst_tag = []
for i in range(len(df7)):
    if df7['tag'][i] != None:
        morph_to_tokens = tokenizer.tokenize(df7['Entity'][i])
        if 1 == len(morph_to_tokens):
            lst_ent.append(df7['Entity'][i])
            lst_tag.append(df7['tag'][i])
            
        else:
            if df7['tag'][i] == 'O':
                lst_ent.append(df7['Entity'][i])
                lst_tag.append(df7['tag'][i])
                for i in range(len(morph_to_tokens)-1):
                    lst_tag.append(df7['tag'][i])
                
            else:
                lst_ent.append(df7['Entity'][i])
                lst_tag.append(df7['tag'][i])
                split = df7['tag'][i].split('-')
                for i in range(len(morph_to_tokens)-1):
                    results = 'I-' + split[-1]
                    lst_tag.append(results)
            
    else:
        lst_ent = result = ' '.join(str(s) for s in lst_ent)
        lst_set.append(lst_ent)
        morph_to_tokens = tokenizer.tokenize(lst_ent)
        lst_set.append(morph_to_tokens)
        lst_set.append(lst_tag)
        lst.append(lst_set)
        result = 0
        lst_ent = []
        lst_tok = []
        lst_tag = []
        lst_set = [] 

In [204]:
print('entity', len(lst[3][0]))
print(lst[3][0])

print('token', len(lst[3][1]))
print(lst[3][1])

print('token_tag', len(lst[3][2]))
print(lst[3][2])

entity 139
29 분 마이클 캐릭 의 침투 패스 를 오른쪽 측면 에서 이어받 은 박지성 은 절묘 한 개인기 로 미들즈브러 수비수 를 제친 뒤 골문 으로 강하 고 정확 한 패스 를 넣 었 고 루니 가 이 를 오른발 슛 으로 연결 , 승부 를 원점 으로 돌렸 다 .
token 83
['29', '분', '마', '##이', '##클', '캐', '##릭', '의', '침', '##투', '패', '##스', '를', '오', '##른', '##쪽', '측', '##면', '에서', '이어', '##받', '은', '박', '##지', '##성', '은', '절', '##묘', '한', '개', '##인', '##기', '로', '미', '##들', '##즈', '##브', '##러', '수', '##비', '##수', '를', '제', '##친', '뒤', '골', '##문', '으로', '강', '##하', '고', '정', '##확', '한', '패', '##스', '를', '넣', '었', '고', '루', '##니', '가', '이', '를', '오', '##른', '##발', '슛', '으로', '연', '##결', ',', '승', '##부', '를', '원', '##점', '으로', '돌', '##렸', '다', '.']
token_tag 83
['B-TI', 'I-TI', 'B-PS', 'I-PS', 'I-PS', 'I-PS', 'I-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PS', 'I-PS', 'I-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

In [164]:
print(lst[3])

['29 분 마이클 캐릭 의 침투 패스 를 오른쪽 측면 에서 이어받 은 박지성 은 절묘 한 개인기 로 미들즈브러 수비수 를 제친 뒤 골문 으로 강하 고 정확 한 패스 를 넣 었 고 루니 가 이 를 오른발 슛 으로 연결 , 승부 를 원점 으로 돌렸 다 .', ['29', '분', '마', '##이', '##클', '캐', '##릭', '의', '침', '##투', '패', '##스', '를', '오', '##른', '##쪽', '측', '##면', '에서', '이어', '##받', '은', '박', '##지', '##성', '은', '절', '##묘', '한', '개', '##인', '##기', '로', '미', '##들', '##즈', '##브', '##러', '수', '##비', '##수', '를', '제', '##친', '뒤', '골', '##문', '으로', '강', '##하', '고', '정', '##확', '한', '패', '##스', '를', '넣', '었', '고', '루', '##니', '가', '이', '를', '오', '##른', '##발', '슛', '으로', '연', '##결', ',', '승', '##부', '를', '원', '##점', '으로', '돌', '##렸', '다', '.'], ['B-TI', 'I-TI', 'B-PS', 'I-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PS', 'O', 'O', 'O', 'O', 'O', 'B-OG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


In [205]:
with open(PATH_preprodata, 'w') as f:
    json.dump(lst, f)

In [206]:
with open(PATH_preprodata, 'r') as f:
    preprodata = json.load(f)

# Example

In [287]:
sent_morph = '29 분 마이클 캐릭 의 침투 패스 를 오른쪽 측면 에서 이어받 은 박지성 은 절묘 한 개인기 로 미들즈브러 수비수 를 제친 뒤 골문 으로 강하 고 정확 한 패스 를 넣 었 고 루니 가 이 를 오른발 슛 으로 연결 , 승부 를 원점 으로 돌렸 다 .'
sent = '29분 마이클 캐릭의 침투 패스를 오른쪽 측면에서 이어받은 박지성은 절묘한 개인기로 미들즈브러 수비수를 제친 뒤 골문으로 강하고 정확한 패스를 넣었고 루니가 이를 오른발 슛으로 연결, 승부를 원점으로 돌렸다.'

In [288]:
morph_to_tokens = tokenizer.tokenize(sent)
print(len(morph_to_tokens))
print(morph_to_tokens)

73
['29', '##분', '마', '##이', '##클', '캐', '##릭', '##의', '침', '##투', '패', '##스를', '오', '##른', '##쪽', '측', '##면', '##에서', '이어', '##받', '##은', '박', '##지', '##성은', '절', '##묘', '##한', '개', '##인', '##기로', '미', '##들', '##즈', '##브', '##러', '수', '##비', '##수를', '제', '##친', '뒤', '골', '##문', '##으로', '강', '##하고', '정', '##확', '##한', '패', '##스를', '넣', '##었고', '루', '##니', '##가', '이를', '오', '##른', '##발', '슛', '##으로', '연', '##결', ',', '승', '##부를', '원', '##점', '##으로', '돌', '##렸다', '.']


In [14]:
sent_to_tokens = tokenizer.tokenize(sent_morph)
print(len(sent_to_tokens))
print(sent_to_tokens)

83
['29', '분', '마', '##이', '##클', '캐', '##릭', '의', '침', '##투', '패', '##스', '를', '오', '##른', '##쪽', '측', '##면', '에서', '이어', '##받', '은', '박', '##지', '##성', '은', '절', '##묘', '한', '개', '##인', '##기', '로', '미', '##들', '##즈', '##브', '##러', '수', '##비', '##수', '를', '제', '##친', '뒤', '골', '##문', '으로', '강', '##하', '고', '정', '##확', '한', '패', '##스', '를', '넣', '었', '고', '루', '##니', '가', '이', '를', '오', '##른', '##발', '슛', '으로', '연', '##결', ',', '승', '##부', '를', '원', '##점', '으로', '돌', '##렸', '다', '.']


# My Data

In [95]:
result = ' '.join(str(s) for s in lst[3][0])

In [96]:
inputs = tokenizer(result, return_tensors='pt')
print(inputs['input_ids'].size())
inputs

torch.Size([1, 85])


{'input_ids': tensor([[   101,  10386,   9367,   9246,  10739, 119327,   9792,  73352,   9637,
           9783,  75855,   9909,  12605,   9233,   9580,  37819,  22028,   9776,
          14867,  24178,  64749, 118965,   9632,   9319,  12508,  17138,   9632,
           9666, 118943,   9954,   8857,  12030,  12310,   9202,   9309,  27023,
          24891,  52015,  30873,   9460,  29455,  15891,   9233,   9672,  55358,
           9109,   8892,  25934,  29805,   8853,  35506,   8888,   9670, 119445,
           9954,   9909,  12605,   9233,   9010,   9557,   8888,   9213,  25503,
           8843,   9638,   9233,   9580,  37819,  51431,   9476,  29805,   9568,
          74322,    117,   9484,  14646,   9233,   9612,  34907,  29805,   9091,
         118881,   9056,    119,    102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 

In [97]:
from transformers import BertModel

In [98]:
PRETAINED_MODEL_NAME = 'bert-base-multilingual-cased'
bert = BertModel.from_pretrained(PRETAINED_MODEL_NAME)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [99]:
outputs = bert(**inputs)

In [100]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [101]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.1460, -0.3585, -0.1289,  ..., -0.0559, -0.0949, -0.0022],
         [ 0.4797, -0.2805, -0.3411,  ...,  0.5483, -0.2645,  0.1313],
         [ 0.2723, -0.2698,  0.2306,  ...,  0.1069,  0.2373, -0.5147],
         ...,
         [-0.4482, -0.5314, -0.1884,  ..., -0.4645,  0.2049, -0.6246],
         [-0.1095, -0.4616, -0.7723,  ..., -0.0311,  0.1242, -0.2750],
         [ 0.2146, -0.3461, -0.3170,  ..., -0.1243,  0.1938, -0.1037]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[ 1.4693e-01, -2.6310e-01,  2.1919e-01, -3.1868e-01, -3.4607e-01,
          3.1139e-02,  1.9270e-01, -3.6483e-02, -1.0906e-01, -1.0840e-01,
         -1.2835e-01,  1.8402e-02, -8.5716e-02,  3.7373e-02,  1.9826e-01,
          1.7237e-01, -1.5535e-01,  1.9489e-01,  1.3541e-01,  6.7832e-02,
         -9.8824e-01, -2.3051e-01,  1.3823e-01, -1.9326e-01,  3.8057e-02,
          2.1562e-01, -3.1980e-01,  1.1865e-01, -7.8593e-03, -3.1718